In [3]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import json
import pyhf
from pyhf.contrib.viz import brazil

import glob

In [21]:
signalList = sorted(glob.glob("./pt*.csv"))[4:] + sorted(glob.glob("./pt*.csv"))[0:4]
signalList

['./pt_miss_histogram_monojet_BMtoy_800GeV.csv',
 './pt_miss_histogram_monojet_BMtoy_850GeV.csv',
 './pt_miss_histogram_monojet_BMtoy_900GeV.csv',
 './pt_miss_histogram_monojet_BMtoy_950GeV.csv',
 './pt_miss_histogram_monojet_BMtoy_1050GeV.csv',
 './pt_miss_histogram_monojet_BMtoy_1100GeV.csv',
 './pt_miss_histogram_monojet_BMtoy_1150GeV.csv',
 './pt_miss_histogram_monojet_BMtoy_1200GeV.csv']

In [22]:
MJData = pd.read_csv("MJData.csv")
MJ_bkg = MJData["MJ_bkg"]
MJ_errbkg = MJData["MJ_errbkg"]
MJ_data = MJData["MJ_data"]
MJ_higgs = MJData["MJ_higgs"]
MJ_axial = MJData["MJ_axial"]

In [27]:
for signal in signalList:
    model1 = pyhf.simplemodels.uncorrelated_background(
        signal=list(pd.read_csv(signal)["counts"]), bkg=list(MJ_bkg), bkg_uncertainty=list(MJ_errbkg)
    ) #creamos el modelo

    observations1 = np.concatenate((MJ_data + pd.read_csv(signal)["counts"], model1.config.auxdata))

    poi_values1 = np.linspace(0.01, 5, 50)
    obs_limit1, exp_limits1, (scan, results) = pyhf.infer.intervals.upper_limits.upper_limit(
        observations1, model1, poi_values1, level=0.05, return_results=True
    )
    pvalue1 = pyhf.infer.hypotest(
        1, observations1, model1, return_expected_set=True, return_tail_probs = True
    )[1][0]
    
    print("Signal " + signal[34:-4])
    print(f"    Upper limit (obs): μ = {obs_limit1:.4f}")
    print(f"    Upper limit (exp): μ = {exp_limits1[2]:.4f} \n")
    print(f"    p-value = {pvalue1:.8f}")